In [1]:
# utilities
import re
import numpy as np
import pandas as pd
import csv

In [2]:
loaded_df=pd.read_csv('flights_cleaned_before_one_hot_encoding_for_cancellation_prediction.csv')
loaded_df.head()

/tmp/ipykernel_3108618/2874402218.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  loaded_df=pd.read_csv('flights_cleaned_before_one_hot_encoding_for_cancellation_prediction.csv')


,MONTH,DAY,DAY_OF_WEEK,AIRLINE,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,...,DIVERTED,SCHEDULED_DEPARTURE_HOUR,SCHEDULED_DEPARTURE_MINUTE,DEPARTURE_TIME_HOUR,DEPARTURE_TIME_MINUTE,DURATION_VARIANCE,IS_PEAK_HOUR,IS_DELAYED,is_holiday,CANCELLATION_REASON_ENCODED
0,1,1,4,AS,ANC,SEA,-0.549427,0.572058,0.841825,0.784874,...,0,0,5,23,54,0.056951,0,0,1,4
1,1,1,4,AA,LAX,PBI,-0.467955,-0.422965,1.839024,1.906734,...,0,0,10,0,2,-0.067709,0,0,1,4
2,1,1,4,US,SFO,CLT,-0.305013,0.019268,1.918800,2.091510,...,0,0,20,0,18,-0.172710,0,0,1,4
3,1,1,4,AA,LAX,MIA,-0.386484,-0.091290,1.905504,1.933130,...,0,0,20,0,15,-0.027626,0,0,1,4
4,1,1,4,AS,SEA,ANC,-0.277856,-0.533523,1.240705,1.062040,...,0,0,25,0,24,0.178665,0,0,1,4


inspect the diverted column.

In [6]:
loaded_df = loaded_df.drop(columns=['DIVERTED'])


In [7]:
loaded_df['CANCELLATION_REASON_ENCODED'].value_counts()

CANCELLATION_REASON_ENCODED
4    5729195
1      48851
0      25262
2      15749
3         22
Name: count, dtype: int64

now, i will try the following things in order:
1. drop encoded =3 (done)
2. oversample 0,1,2 to 100k and undersample 4 to 100k
3. mix 0,1,2,3 as cancelled and 4 as not cancelled, go for binary classification.

common things:
one hot encoding on airline columns.
freq_encoding on origin and desti airport, measure freq on the whole dataset, then apply after train, test split.


In [8]:
df_sampled2 = pd.get_dummies(loaded_df, columns=["AIRLINE"], prefix="AIRLINE", drop_first=True)

# Only convert the one-hot encoded columns to int
airline_columns = [col for col in df_sampled2.columns if col.startswith("AIRLINE_")]
df_sampled2[airline_columns] = df_sampled2[airline_columns].astype(int)

In [10]:
df_sampled2 = df_sampled2[df_sampled2["CANCELLATION_REASON_ENCODED"] != 3]

In [3]:
df_sampled2['CANCELLATION_REASON_ENCODED'].value_counts()

CANCELLATION_REASON_ENCODED
4    5729195
1      48851
0      25262
2      15749
Name: count, dtype: int64

In [12]:
df_sampled2.to_csv("cancel_predict_2_after_one_hot.csv")

In [2]:
df_sampled2=pd.read_csv('cancel_predict_2_after_one_hot.csv')
df_sampled2.head()

/tmp/ipykernel_1373715/3272129661.py:1: DtypeWarning: Columns (4,5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_sampled2=pd.read_csv('cancel_predict_2_after_one_hot.csv')


,Unnamed: 0,MONTH,DAY,DAY_OF_WEEK,ORIGIN_AIRPORT,DESTINATION_AIRPORT,DEPARTURE_DELAY,TAXI_OUT,SCHEDULED_TIME,ELAPSED_TIME,...,AIRLINE_EV,AIRLINE_F9,AIRLINE_HA,AIRLINE_MQ,AIRLINE_NK,AIRLINE_OO,AIRLINE_UA,AIRLINE_US,AIRLINE_VX,AIRLINE_WN
0,0,1,1,4,ANC,SEA,-0.549427,0.572058,0.841825,0.784874,...,0,0,0,0,0,0,0,0,0,0
1,1,1,1,4,LAX,PBI,-0.467955,-0.422965,1.839024,1.906734,...,0,0,0,0,0,0,0,0,0,0
2,2,1,1,4,SFO,CLT,-0.305013,0.019268,1.918800,2.091510,...,0,0,0,0,0,0,0,1,0,0
3,3,1,1,4,LAX,MIA,-0.386484,-0.091290,1.905504,1.933130,...,0,0,0,0,0,0,0,0,0,0
4,4,1,1,4,SEA,ANC,-0.277856,-0.533523,1.240705,1.062040,...,0,0,0,0,0,0,0,0,0,0


In [27]:
from sklearn.model_selection import train_test_split

X = df_sampled2.drop("CANCELLATION_REASON_ENCODED", axis=1)  # Features
y = df_sampled2["CANCELLATION_REASON_ENCODED"]

# Split into train+validation and test sets
X_train_val, X_test, y_train_val, y_test = train_test_split(
    X, y, test_size=0.05, random_state=42
)

# Split train+validation into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(
    X_train_val, y_train_val, test_size=0.05555, random_state=42  # 0.1765 = 10 / 90
)

print(f"Train set: {X_train.shape}, Validation set: {X_val.shape}, Test set: {X_test.shape}")

Train set: (5221017, 35), Validation set: (307087, 35), Test set: (290953, 35)


# sampling

# freq encoding part


In [23]:
# Compute frequency maps
origin_airport_freq_map = df_sampled2["ORIGIN_AIRPORT"].value_counts().to_dict()
print(origin_airport_freq_map)

{'ATL': 346836, 'ORD': 285884, 'DFW': 239551, 'DEN': 196055, 'LAX': 194673, 'SFO': 148007, 'PHX': 146815, 'IAH': 146622, 'LAS': 133180, 'MSP': 112117, 'MCO': 110980, 'SEA': 110899, 'DTW': 108500, 'BOS': 107847, 'EWR': 101772, 'CLT': 100324, 'LGA': 99604, 'SLC': 97210, 'JFK': 93811, 'BWI': 86079, 'MDW': 80886, 'DCA': 73794, 'FLL': 73624, 'SAN': 70201, 'MIA': 69339, 'PHL': 66021, 'TPA': 63905, 'DAL': 59699, 'HOU': 52041, 'BNA': 47580, 'PDX': 47085, 'STL': 46963, 'HNL': 43172, 'OAK': 42316, 'AUS': 42067, 'MSY': 38801, 'MCI': 38665, 'SJC': 38200, 'SMF': 37212, 'SNA': 37186, 'CLE': 34589, 'IAD': 34305, 'RDU': 31896, 10397: 31883, 'MKE': 29824, 'SAT': 29788, 'RSW': 27350, 13930: 27017, 'IND': 25558, 'SJU': 24655, 'CMH': 24187, 'PIT': 24118, 'PBI': 22573, 'OGG': 20627, 11298: 20573, 'CVG': 20031, 'ABQ': 19192, 'BUR': 18889, 'BDL': 18799, 'JAX': 18038, 'ONT': 17978, 11292: 17741, 12892: 17310, 'BUF': 16824, 'OMA': 16753, 'OKC': 16144, 'ANC': 16005, 'RIC': 15355, 'TUS': 15117, 'MEM': 14770, 'TU

In [24]:
destination_airport_freq_map = df_sampled2["DESTINATION_AIRPORT"].value_counts().to_dict()
print(destination_airport_freq_map)

{'ATL': 346904, 'ORD': 285906, 'DFW': 239582, 'DEN': 196009, 'LAX': 194696, 'SFO': 147966, 'PHX': 146811, 'IAH': 146682, 'LAS': 133195, 'MSP': 112128, 'MCO': 110978, 'SEA': 110907, 'DTW': 108398, 'BOS': 107851, 'EWR': 101830, 'CLT': 100322, 'LGA': 99580, 'SLC': 97193, 'JFK': 93809, 'BWI': 86084, 'MDW': 80884, 'DCA': 73798, 'FLL': 73598, 'SAN': 70206, 'MIA': 69322, 'PHL': 66017, 'TPA': 63913, 'DAL': 59703, 'HOU': 52040, 'BNA': 47568, 'PDX': 47081, 'STL': 46962, 'HNL': 43157, 'OAK': 42312, 'AUS': 42078, 'MSY': 38800, 'MCI': 38655, 'SJC': 38203, 'SMF': 37212, 'SNA': 37194, 'CLE': 34580, 'IAD': 34282, 'RDU': 31897, 10397: 31880, 'MKE': 29833, 'SAT': 29784, 'RSW': 27349, 13930: 26991, 'IND': 25557, 'SJU': 24653, 'CMH': 24183, 'PIT': 24121, 'PBI': 22574, 'OGG': 20638, 11298: 20561, 'CVG': 20027, 'ABQ': 19188, 'BUR': 18890, 'BDL': 18797, 'JAX': 18040, 'ONT': 17979, 11292: 17708, 12892: 17303, 'BUF': 16829, 'OMA': 16752, 'OKC': 16147, 'ANC': 16001, 'RIC': 15351, 'TUS': 15116, 'MEM': 14769, 'TU

In [25]:
print(len(origin_airport_freq_map))

print(len(destination_airport_freq_map))

925
920


In [26]:
# Combine two dictionaries, prioritizing origin_airport_freq_map for overlapping keys
combined_airport_freq_map = {**destination_airport_freq_map, **origin_airport_freq_map}

# Display the combined dictionary
print(len(combined_airport_freq_map))
print(combined_airport_freq_map)


927
{'ATL': 346836, 'ORD': 285884, 'DFW': 239551, 'DEN': 196055, 'LAX': 194673, 'SFO': 148007, 'PHX': 146815, 'IAH': 146622, 'LAS': 133180, 'MSP': 112117, 'MCO': 110980, 'SEA': 110899, 'DTW': 108500, 'BOS': 107847, 'EWR': 101772, 'CLT': 100324, 'LGA': 99604, 'SLC': 97210, 'JFK': 93811, 'BWI': 86079, 'MDW': 80886, 'DCA': 73794, 'FLL': 73624, 'SAN': 70201, 'MIA': 69339, 'PHL': 66021, 'TPA': 63905, 'DAL': 59699, 'HOU': 52041, 'BNA': 47580, 'PDX': 47085, 'STL': 46963, 'HNL': 43172, 'OAK': 42316, 'AUS': 42067, 'MSY': 38801, 'MCI': 38665, 'SJC': 38200, 'SMF': 37212, 'SNA': 37186, 'CLE': 34589, 'IAD': 34305, 'RDU': 31896, 10397: 31883, 'MKE': 29824, 'SAT': 29788, 'RSW': 27350, 13930: 27017, 'IND': 25558, 'SJU': 24655, 'CMH': 24187, 'PIT': 24118, 'PBI': 22573, 'OGG': 20627, 11298: 20573, 'CVG': 20031, 'ABQ': 19192, 'BUR': 18889, 'BDL': 18799, 'JAX': 18038, 'ONT': 17978, 11292: 17741, 12892: 17310, 'BUF': 16824, 'OMA': 16753, 'OKC': 16144, 'ANC': 16005, 'RIC': 15355, 'TUS': 15117, 'MEM': 14770,

In [28]:
# Frequency encoding for train, validation, and test sets
X_train["ORIGIN_AIRPORT_freq"] = X_train["ORIGIN_AIRPORT"].map(combined_airport_freq_map)
X_val["ORIGIN_AIRPORT_freq"] = X_val["ORIGIN_AIRPORT"].map(combined_airport_freq_map)
X_test["ORIGIN_AIRPORT_freq"] = X_test["ORIGIN_AIRPORT"].map(combined_airport_freq_map)

X_train["DESTINATION_AIRPORT_freq"] = X_train["DESTINATION_AIRPORT"].map(combined_airport_freq_map)
X_val["DESTINATION_AIRPORT_freq"] = X_val["DESTINATION_AIRPORT"].map(combined_airport_freq_map)
X_test["DESTINATION_AIRPORT_freq"] = X_test["DESTINATION_AIRPORT"].map(combined_airport_freq_map)

In [29]:
X_train.isnull().sum()

Unnamed: 0                    0
MONTH                         0
DAY                           0
DAY_OF_WEEK                   0
ORIGIN_AIRPORT                0
DESTINATION_AIRPORT           0
DEPARTURE_DELAY               0
TAXI_OUT                      0
SCHEDULED_TIME                0
ELAPSED_TIME                  0
AIR_TIME                      0
DISTANCE                      0
TAXI_IN                       0
SCHEDULED_ARRIVAL             0
SCHEDULED_DEPARTURE_HOUR      0
SCHEDULED_DEPARTURE_MINUTE    0
DEPARTURE_TIME_HOUR           0
DEPARTURE_TIME_MINUTE         0
DURATION_VARIANCE             0
IS_PEAK_HOUR                  0
IS_DELAYED                    0
is_holiday                    0
AIRLINE_AS                    0
AIRLINE_B6                    0
AIRLINE_DL                    0
AIRLINE_EV                    0
AIRLINE_F9                    0
AIRLINE_HA                    0
AIRLINE_MQ                    0
AIRLINE_NK                    0
AIRLINE_OO                    0
AIRLINE_

In [30]:
# Drop the original airport code columns
X_train = X_train.drop(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1)
X_val = X_val.drop(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1)
X_test = X_test.drop(["ORIGIN_AIRPORT", "DESTINATION_AIRPORT"], axis=1)

In [34]:
y_test.value_counts()

CANCELLATION_REASON_ENCODED
4    286509
1      2378
0      1274
2       792
Name: count, dtype: int64

In [31]:
from imblearn.over_sampling import SMOTE
from imblearn.under_sampling import RandomUnderSampler
from imblearn.pipeline import Pipeline
from collections import Counter

# Initial class distribution
print(f"Original class distribution: {Counter(y_train)}")

# Define oversampling and undersampling strategies
oversample_strategy = {0: 200000, 1: 200000, 2: 200000}  # Oversample minority classes to 100k
undersample_strategy = {0: 200000, 1: 200000, 2: 200000, 4: 200000}  # Undersample majority class to 100k

# Create a pipeline with SMOTE for oversampling and RandomUnderSampler for undersampling
pipeline = Pipeline([
    ('oversample', SMOTE(sampling_strategy=oversample_strategy, random_state=42)),
    ('undersample', RandomUnderSampler(sampling_strategy=undersample_strategy, random_state=42))
])

# Apply the pipeline to the data
X_train_resampled, y_train_resampled = pipeline.fit_resample(X_train, y_train)

# Check the new class distribution
print(f"Resampled class distribution before: {Counter(y_train)}")

# Check the new class distribution
print(f"Resampled class distribution after: {Counter(y_train_resampled)}")

Original class distribution: Counter({4: 5140470, 1: 43820, 0: 22593, 2: 14134})
Resampled class distribution before: Counter({4: 5140470, 1: 43820, 0: 22593, 2: 14134})
Resampled class distribution after: Counter({0: 200000, 1: 200000, 2: 200000, 4: 200000})


In [39]:
X_train_resampled.to_csv("X_train_resampled.csv", index=False)
y_train_resampled.to_csv("y_train_resampled.csv", index=False)
X_val.to_csv("X_val.csv", index=False)
y_val.to_csv("y_val.csv", index=False)
X_test.to_csv("X_test.csv", index=False)
y_test.to_csv("y_test.csv", index=False)

In [40]:
X_train_resampled.info()

<class 'pandas.core.frame.DataFrame'>
Index: 800000 entries, 5318161 to 1120018
Data columns (total 35 columns):
 #   Column                      Non-Null Count   Dtype  
---  ------                      --------------   -----  
 0   Unnamed: 0                  800000 non-null  int64  
 1   MONTH                       800000 non-null  int64  
 2   DAY                         800000 non-null  int64  
 3   DAY_OF_WEEK                 800000 non-null  int64  
 4   DEPARTURE_DELAY             800000 non-null  float64
 5   TAXI_OUT                    800000 non-null  float64
 6   SCHEDULED_TIME              800000 non-null  float64
 7   ELAPSED_TIME                800000 non-null  float64
 8   AIR_TIME                    800000 non-null  float64
 9   DISTANCE                    800000 non-null  float64
 10  TAXI_IN                     800000 non-null  float64
 11  SCHEDULED_ARRIVAL           800000 non-null  int64  
 12  SCHEDULED_DEPARTURE_HOUR    800000 non-null  int64  
 13  SCHEDULED_DE

# training part

# Evaluation from Baseline Models

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, accuracy_score
import pickle



#baseline_model = LogisticRegression(multi_class="multinomial", solver="lbfgs", max_iter=500, random_state=42)
#penalty='l2', C=0.1
baseline_model = LogisticRegression(
    multi_class="multinomial", solver="lbfgs", max_iter=1000, random_state=42, class_weight="balanced", penalty='l2', C=0.1
)
baseline_model.fit(X_train, y_train)

# Evaluate baseline model on validation set
y_val_pred_baseline = baseline_model.predict(X_val)
print("Baseline Model Accuracy (Validation):", accuracy_score(y_val, y_val_pred_baseline))
print(classification_report(y_val, y_val_pred_baseline, target_names={'A': 0, 'B': 1, 'C': 2, 'D': 3, 'N': 4}))



# Save the trained model to a file
with open("trained_model.pkl", "wb") as f:
    pickle.dump(baseline_model, f)
